In [3]:
import math
import networkx as nx
import db, pathSearch
points = [[35.16487131336672, 136.9374775886536], [35.16378437005421, 136.9397842884064], [35.16297791885458, 136.9374775886536]]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)
link, length = db.getRectangleRoadData(y1, x1, y2, x2)
G = pathSearch.linkToGraph(link, length)
length = dict(nx.all_pairs_dijkstra_path_length(G))
print(length['[35.1635255, 136.9400303]'])

{'[35.1635255, 136.9400303]': 0, '[35.1636073, 136.9394786]': 0.05096947, '[35.1634227, 136.9394362]': 0.071854803, '[35.1640066, 136.9395699]': 0.09613872300000001, '[35.1629381, 136.9393217]': 0.126735943, '[35.164399, 136.9396591]': 0.14051861300000001, '[35.1637556, 136.938429]': 0.14779575, '[35.1635797, 136.9383886]': 0.16769641700000001, '[35.1647606, 136.9397392]': 0.18138073300000002, '[35.1634256, 136.9383532]': 0.18513120000000002, '[35.1643158, 136.9402128]': 0.19169428600000002, '[35.1641528, 136.9385202]': 0.19273375, '[35.1630828, 136.9382744]': 0.22328948299999998, '[35.1651538, 136.9398408]': 0.226067333, '[35.1645473, 136.9386028]': 0.237238057, '[35.1639101, 136.9373535]': 0.24705967, '[35.1627501, 136.9381986]': 0.260920283, '[35.1649102, 136.9386832]': 0.27824724700000003, '[35.163582, 136.9372752]': 0.284656085, '[35.1643126, 136.9374495]': 0.291671634, '[35.1632437, 136.9371944]': 0.323082239, '[35.1647013, 136.937521]': 0.33537900400000004, '[35.1650675, 136.937

In [4]:
p, le,_ = pathSearch.travelingPath(points, link, length)
print(p)
print(le)
print(y1, x1, y2, x2)

[[35.1647013, 136.937521], [35.1643126, 136.9374495], [35.1641528, 136.9385202], [35.1640066, 136.9395699], [35.1640066, 136.9395699], [35.1641528, 136.9385202], [35.1637556, 136.938429], [35.1639101, 136.9373535], [35.163582, 136.9372752], [35.1632437, 136.9371944], [35.1629191, 136.9371148], [35.1629191, 136.9371148], [35.1632437, 136.9371944], [35.163582, 136.9372752], [35.1639101, 136.9373535], [35.1643126, 136.9374495], [35.1647013, 136.937521]]
0.79436472
35.16487131336672 136.9374775886536 35.16297791885458 136.9397842884064


In [2]:
G = pathSearch.linkToGraph(link, length)
print(list(G.nodes))

['[35.1624723, 136.9403682]', '[35.1626084, 136.9392426]', '[35.1657816, 136.9399919]', '[35.1651538, 136.9398408]', '[35.1653085, 136.9387857]', '[35.1652278, 136.9393279]', '[35.1634444, 136.940595]', '[35.1635255, 136.9400303]', '[35.1636073, 136.9394786]', '[35.1645473, 136.9386028]', '[35.164399, 136.9396591]', '[35.1643158, 136.9402128]', '[35.1642346, 136.9407811]', '[35.1628586, 136.9398764]', '[35.1649102, 136.9386832]', '[35.1647606, 136.9397392]', '[35.1658611, 136.9394589]', '[35.1634194, 136.936091]', '[35.1632437, 136.9371944]', '[35.1630828, 136.9382744]', '[35.1629381, 136.9393217]', '[35.1627768, 136.9404377]', '[35.1641528, 136.9385202]', '[35.1637556, 136.938429]', '[35.1635797, 136.9383886]', '[35.1634256, 136.9383532]', '[35.1627501, 136.9381986]', '[35.1640066, 136.9395699]', '[35.1634227, 136.9394362]', '[35.1618272, 136.9390696]', '[35.1625422, 136.9370223]', '[35.1629191, 136.9371148]', '[35.163582, 136.9372752]', '[35.1639101, 136.9373535]', '[35.1643126, 136.

In [3]:
tsp = ['[35.1647013, 136.937521]', '[35.1640066, 136.9395699]', '[35.1629191, 136.9371148]', '[35.1647013, 136.937521]']
p = tsp[0]
rp = p.strip("[]")
print(p)
print(rp)

[35.1647013, 136.937521]
35.1647013, 136.937521


In [2]:
import csv
import networkx as nx
import db, pathSearch

# データを用意
points = [[35.16966936723668, 136.90265178680423], [35.15224248922456, 136.95080280303958]]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)
link, length = db.getRectangleRoadData(y1, x1, y2, x2)
G = pathSearch.linkToGraph(link, length)
length = dict(nx.all_pairs_dijkstra_path_length(G))
# CSVファイルにデータを書き込む
with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')  # タブを区切り文字として指定
    for u in length.keys():
        for v in length[u].keys():
            data = [u, v, length[u][v]]
            writer.writerow(data)


In [2]:
import csv

dic = {}
with open('data.csv', 'r', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    l = [row for row in reader]
    for elem in l:
        if elem[0] in dic.keys():
            dic[elem[0]][elem[1]] = elem[2]
        else:
            dic[elem[0]] = {elem[1] : elem[2]}
            
print(dic['[35.1635255, 136.9400303]']['[35.1636073, 136.9394786]'])

0.05096947


In [1]:
nodes = []
def dist():
    return
def swap():
    return

while(True):
    Flag = True
    for p in nodes:
        for q in nodes:
            d1 = dist(p, q)
            d2 = dist(p+1, q+1)
            d3 = dist(p, p+1)
            d4 = dist(q, q+1)
            if d1+d2 < d3+d4:
                swap(p, q)
                Flag = False
    if Flag:
        break


False


In [39]:
import pathSearch, db
import random, time
import numpy as np
from scipy import stats

N = 50
moveDist = 1000
ite = 20

def generate_random_points_in_rectangle(rectangle, n):
    x_min, y_min = min(rectangle[0][0], rectangle[1][0]), min(rectangle[0][1], rectangle[1][1])
    x_max, y_max = max(rectangle[0][0], rectangle[1][0]), max(rectangle[0][1], rectangle[1][1])
    random_points = [[random.uniform(x_min, x_max), random.uniform(y_min, y_max)] for _ in range(n)]
    return random_points

coordinates = [[35.18441016255083, 136.9190883636475], [35.15573127979675, 136.97341918945315]]

sum_len_old = 0
sum_len_walk_old = 0
sum_len_new = 0
sum_len_walk_new = 0
sum_time_old = 0
sum_time_new = 0
count_old = 0
count_new = 0
t_test_old = []
t_test_old_walk = []
t_test_new = []
t_test_new_walk = []
for i in range(ite):
    points = generate_random_points_in_rectangle(coordinates, N)
    y1, x1, y2, x2 = pathSearch.rectangleArea(points)
    try:
        link, length = db.getRectangleRoadData(y1, x1, y2, x2, 1.25)

        start = time.perf_counter()
        _, len_old, _, _, _, len_walk_old = pathSearch.sharedRidePath(points, link, length, moveDist, "type4", len_dic)
        end = time.perf_counter()
        sum_time_old += end - start
        sum_len_old += len_old
        sum_len_walk_old += len_walk_old
        count_old += 1
        t_test_old.append(len_old)
        t_test_old_walk.append(len_walk_old)

        start = time.perf_counter()
        _, len_new, _, _, _, len_walk_new = pathSearch.sharedRidePath(points, link, length, moveDist, "type2", len_dic)
        end = time.perf_counter()
        sum_time_new += end - start
        sum_len_new += len_new
        sum_len_walk_new += len_walk_new
        count_new += 1
        t_test_new.append(len_new)
        t_test_new_walk.append(len_walk_new)

    except:
        continue
ave_len_old = sum_len_old / count_old
ave_len_walk_old = sum_len_walk_old / count_old
ave_len_new = sum_len_new / count_new
ave_len_walk_new = sum_len_walk_new / count_new
ratio = (ave_len_old - ave_len_new) / ave_len_old
person = (ave_len_walk_old - ave_len_walk_new) / N
ave_time_old = sum_time_old / count_old
ave_time_new = sum_time_new / count_new
print(ave_len_old)
print(ave_len_walk_old)
print(ave_len_new)
print(ave_len_walk_new)
print(ratio*100)
print(person*1000)
print(count_old,count_new)
print(ave_time_old, ave_time_new)

if count_new == count_old:
    print(stats.ttest_rel(t_test_old, t_test_new))
    print(stats.ttest_rel(t_test_old_walk, t_test_new_walk))


12.903891148337491
34.600611640900006
12.6813294042125
28.26231253823125
1.72476458121445
126.76598205337514
16 16
15.986185343750321 15.560754406249998
TtestResult(statistic=2.2665087032973, pvalue=0.03864164135360939, df=15)
TtestResult(statistic=13.583185623456872, pvalue=7.80986847888958e-10, df=15)


In [1]:
import networkx as nx
import pathSearch, db
points = [[35.201031449938064, 136.89788818359378], [35.14030839946502, 136.9981384277344]]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)
link, length = db.getRectangleRoadData(y1, x1, y2, x2)
G = pathSearch.linkToGraph(link, length)
len_dic = dict(nx.all_pairs_dijkstra_path_length(G))
print("---data loaded---")

---data loaded---
